In [ ]:
import pandas as pd
import numpy as np

dfData = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/sample_files/master/workshop5.csv', delimiter=',')


In [ ]:
dfData.head()

In [ ]:
set1Filter = 'SH.XPD.CHEX.PC.CD'
set1Label = 'Current health expenditure per capita (current US$)'
#set2Filter = 'SH.DYN.NCOM.ZS'
#set2Label = 'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)'
set2Filter = 'SP.DYN.LE00.IN'
set2Label = 'Life expectancy at birth, total (years)'

dset = pd.DataFrame(index = ['Country Code', 'Set1','Set2', 'Year'])

dset

In [ ]:
for i in range(1960,2019):
    yearFilter = str(i)
    
    dset1 = dfData[(dfData['Indicator Code'] == set1Filter)][['Country Code', yearFilter]]
    dset1.rename(columns={yearFilter:'Set1'}, inplace = True)


    dset2 = dfData[(dfData['Indicator Code'] == set2Filter)][['Country Code', yearFilter]]
    dset2.rename(columns={yearFilter:'Set2'}, inplace = True)

    dsetOut = pd.merge(dset1,dset2, how = 'inner', on = ['Country Code'])
    dsetOut['Year'] = yearFilter
    dsetOut.dropna(inplace = True)

    dset = dset.append(dsetOut, ignore_index = True, sort = False)

dset = dset.dropna()
#dset['Year'].astype('int')

In [ ]:
dset.info()

In [ ]:
set1Range = dset['Set1'].agg([np.min, np.max])
set2Range = dset['Set2'].agg([np.min, np.max])
yearRange = dset['Year'].agg([np.min, np.max])

yearRange.astype('int')

print(set1Range)
print(set2Range)
print(yearRange)


In [ ]:
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.layouts import widgetbox, column
from bokeh.models import CustomJS, ColumnDataSource, Slider

#output_notebook()
output_notebook()

source1 = ColumnDataSource(dset[dset['Year'] == '2000'])
source = ColumnDataSource(dset)

p = figure(plot_width=900, plot_height=600, title = set2Label + ' by ' + set1Label,
          x_axis_label= set1Label, y_axis_label = set2Label,
          x_range = set1Range, y_range = set2Range)

p.circle(x='Set1', y='Set2', size=15, color = 'blue', alpha=0.7, source = source1)

show(p)

In [ ]:
# Create first slider: slider1
slider = Slider(title='Year',start=int(yearRange['amin']),end=int(yearRange['amax']),step=5,value=int(yearRange['amin']))
# Add slider1 and slider2 to a widgetbox
layout = column(widgetbox(slider),p)


In [ ]:

def data_change():
    return df['x','y']

def sliderChange(attr, old, new):
    source = ColumnDataSource(df[df['z'] == slider.value])
    print(slider.value)
    
sliderChange = CustomJS(args=dict(source1=source1, source=source, slider=slider), code="""
    var data1 = source1.data;
    var data = source.data;
    var f = slider.value;

    data1['Set1'] = [];
    data1['Set2'] = [];
    data1['Year'] = [];

    console.log(data1);
    for (i in data['Year']) {
        if (data['Year'][i] == slider.value){
            console.log(data['Year'][i]);
            data1['Set1'].push(data['Set1'][i]);
            data1['Set2'].push(data['Set2'][i]);
            data1['Year'].push(data['Year'][i]);
        }
        
    }
    
    // necessary becasue we mutated source.data in-place
    source1.change.emit();
""")

slider.js_on_change('value', sliderChange)


In [ ]:
show(layout)